## <font color='green'> <div align="center">In the name of God </div></font>

### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>

# GMM Quadratic Classifier on MNIST with reg = 0.1

##### Importing general modules.

In [1]:
import numpy as np
from sklearn import mixture
import scipy.io as sio
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

### Preparing matplotlib to genrate high-quality images for the paper

In [2]:
%matplotlib inline
mpl.rcParams['figure.dpi']= 600

In [3]:
C  = 10
K = 6
L = C * K
width = 28
height = 28
reg = 0.1

### Load Training and Testing Data and Normalize them

In [4]:
XTrain = sio.loadmat ('../../datasets/mnist/MnistTrainX')['MnistTrainX']
XTrain = XTrain / 255
yTrain = sio.loadmat ('../../datasets/mnist/MnistTrainY')['MnistTrainY']
XTest = sio.loadmat ('../../datasets/mnist/MnistTestX')['MnistTestX']
XTest = XTest / 255
yTest = sio.loadmat ('../../datasets/mnist/MnistTestY')['MnistTestY']
N, dim = XTrain.shape

### Loading GMM data computed on Google Colab

In [5]:
clusters = np.zeros(N, dtype = int)
with open('GMM_MNIST_6.txt', 'r') as f:
    for i in range (N):
        s = f.readline()
        clusters[i] = int(s)

### Compute means and covariances

In [6]:
means = np.zeros([L, dim])
cov = np.zeros([L, dim, dim])
for i in range (L):
    idx = clusters == i
    means[i,:] = np.mean(XTrain[idx,:], axis = 0)
    cov[i,:,:] = np.cov (XTrain[idx,:].T)    

### Compute priors

In [9]:
Nck = np.bincount(clusters)
Pck = Nck / N

In [10]:
logProb = np.zeros ([N,L])
for i in tqdm(range (L)):
    eigvals, eigvecs = np.linalg.eig(cov[i,:,:])
    eigvals = eigvals + reg
    sigmaHalfInv = eigvecs @ np.diag(np.sqrt(1/eigvals)) @ eigvecs.T
    X = (XTrain - means[i,:]) @ sigmaHalfInv
    X = X ** 2
    d2 = np.sum(X, axis = 1)
    logProb[:,i] = np.log(Pck[i]) - 1 / 2 * d2
prediction = np.argmax (logProb, axis = 1) // K
score = np.sum(prediction == yTrain.squeeze())/ N
print ('Training Accuracy = ', score)


  0%|                                                                                           | 0/60 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.

100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [12:38<00:00, 14.21s/it]

Training Accuracy =  0.9815666666666667


In [11]:
NTest = XTest.shape[0]
logProb = np.zeros ([NTest,L])
for i in tqdm(range (L)):
    eigvals, eigvecs = np.linalg.eig(cov[i,:,:])
    eigvals = eigvals + reg
    sigmaHalfInv = eigvecs @ np.diag(np.sqrt(1/eigvals)) @ eigvecs.T
    X = (XTest - means[i,:]) @ sigmaHalfInv
    X = X ** 2
    d2 = np.sum(X, axis = 1)
    logProb[:,i] = np.log(Pck[i]) - 1 / 2 * d2
prediction = np.argmax (logProb, axis = 1) // K
score = np.sum(prediction == yTest.squeeze())/ NTest
print ('Testing Accuracy = ', score)


  0%|                                                                                           | 0/60 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  # This is added back by InteractiveShellApp.init_path()

100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [02:53<00:00,  3.30s/it]

Testing Accuracy =  0.9701


### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>